In [4]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score


In [5]:
!pip show nltk


Name: nltk
Version: 3.8.1
Summary: Natural Language Toolkit
Home-page: https://www.nltk.org/
Author: NLTK Team
Author-email: nltk.team@gmail.com
License: Apache License, Version 2.0
Location: C:\ProgramData\anaconda3\Lib\site-packages
Requires: click, joblib, regex, tqdm
Required-by: textblob


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [6]:
!pip install nltk


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [9]:
# Load the dataset
df = pd.read_csv(r"C:\Users\sruth\Downloads\Fraud Analytics_Project\GPT-wiki-intro\GPT-wiki-intro.csv")

# Display the first few rows and check for columns that might indicate human or AI-generated text
print(df.head())
print(df.columns)

         id                                                url  \
0  63064638  https://en.wikipedia.org/wiki/Sexhow%20railway...   
1    279621        https://en.wikipedia.org/wiki/Eti%C3%A4inen   
2    287229  https://en.wikipedia.org/wiki/Inverse%20functi...   
3  26712375  https://en.wikipedia.org/wiki/Stepping%20on%20...   
4  38894426        https://en.wikipedia.org/wiki/Rob%20Bradley   

                      title  \
0    Sexhow railway station   
1                  Etiäinen   
2  Inverse function theorem   
3         Stepping on Roses   
4               Rob Bradley   

                                          wiki_intro  \
0  Sexhow railway station was a railway station b...   
1  In Finnish folklore, all places and things, an...   
2  In mathematics, specifically differential calc...   
3  is a Japanese shōjo manga series written and i...   
4  Robert Milner "Rob" Bradley, Jr. (born August ...   

                                     generated_intro  title_len  \
0  Sexhow ra

In [10]:
df

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,44173767,https://en.wikipedia.org/wiki/Randy%20Borum,Randy Borum,Randy Borum is a Professor and Coordinator of ...,Randy Borum is a Professor and Coordinator of ...,2,185,71,200 word wikipedia style introduction on 'Rand...,of the Master of Fine Arts Program in Creativ...,25,92
149996,33564134,https://en.wikipedia.org/wiki/Sa%27och%20language,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nea...","Sa'och (, also, ""Sauch"") is an endangered, nuc...",2,175,134,200 word wikipedia style introduction on 'Sa'o...,"nuclear-speaking, isolate language of the Ath...",33,184
149997,4219548,https://en.wikipedia.org/wiki/Philip%20Hanawalt,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,Philip C. Hanawalt (born 1931) is an American ...,2,166,191,200 word wikipedia style introduction on 'Phil...,American graphic artist and illustrator. He i...,30,272
149998,2625970,https://en.wikipedia.org/wiki/Vossius%20Gymnasium,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in Ams...,Vossius Gymnasium is a public gymnasium in the...,2,168,108,200 word wikipedia style introduction on 'Voss...,"the town of Vossius, Netherlands. It is named...",32,147


### Labelling and Re-structuring the data

In [11]:
# Human-generated text
human_texts = df[['id', 'title', 'wiki_intro']].copy()
human_texts['label'] = 'HGT'  # Human Generated Text
human_texts.rename(columns={'wiki_intro': 'text'}, inplace=True)  # Rename column to 'text'

# Machine-generated text
machine_texts = df[['id', 'title', 'generated_intro']].copy()
machine_texts['label'] = 'MGT'  # Machine Generated Text
machine_texts.rename(columns={'generated_intro': 'text'}, inplace=True)  # Rename column to 'text'

# Combine both into a single DataFrame
combined_df = pd.concat([human_texts, machine_texts], ignore_index=True)

# Inspect the resulting DataFrame
print(combined_df.head())

         id                     title  \
0  63064638    Sexhow railway station   
1    279621                  Etiäinen   
2    287229  Inverse function theorem   
3  26712375         Stepping on Roses   
4  38894426               Rob Bradley   

                                                text label  
0  Sexhow railway station was a railway station b...   HGT  
1  In Finnish folklore, all places and things, an...   HGT  
2  In mathematics, specifically differential calc...   HGT  
3  is a Japanese shōjo manga series written and i...   HGT  
4  Robert Milner "Rob" Bradley, Jr. (born August ...   HGT  


In [12]:
mgt_rows = combined_df[combined_df['label'] == 'MGT']
mgt_rows

,id,title,text,label
150000,63064638,Sexhow railway station,Sexhow railway station was a railway station l...,MGT
150001,279621,Etiäinen,"In Finnish folklore, all places and things, an...",MGT
150002,287229,Inverse function theorem,"In mathematics, specifically differential calc...",MGT
150003,26712375,Stepping on Roses,is a Japanese shōjo manga series written and i...,MGT
150004,38894426,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...",MGT
...,...,...,...,...
299995,44173767,Randy Borum,Randy Borum is a Professor and Coordinator of ...,MGT
299996,33564134,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nuc...",MGT
299997,4219548,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,MGT
299998,2625970,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in the...,MGT


### Text Pre-processing

In [13]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sruth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sruth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # Tokenizing
    words = word_tokenize(text)
    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

combined_df['cleaned_text'] = combined_df['text'].apply(preprocess_text)

### Split dataset into training and test sets

In [ ]:
X = combined_df['cleaned_text']
y = combined_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Feature Extraction

In [ ]:
# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=5000)  # You can adjust the max_features based on your needs

# Fit and transform the training data
X_train_tfidf = tfidf.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf.transform(X_test)

#### Use X_train_tfidf and X_test_tfidf for the model building

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [ ]:
# Define the specified models
models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'Decision Tree': DecisionTreeClassifier(max_depth=5),
    'Random Forest': RandomForestClassifier(n_estimators=50, max_depth=5),
}

# Evaluate each model using a pipeline
for name, model in models.items():
    print(f"\n--- {name} ---")
    
    # Create a pipeline with TfidfVectorizer and the chosen model
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=500)),  # Use TF-IDF for feature extraction
        ('classifier', model)
    ])
    
    # Train the model
    pipeline.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = pipeline.predict(X_test)
    
    # Evaluate the model
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

### NLP Techniques 

### Named-Entity Recognition: 
NER is an NLP technique that identifies and categorizes entities (specific elements) in text into predefined categories such as names of persons, organizations, locations, dates, times, quantities, monetary values, and more.

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy

# Load the pre-trained NLP model
nlp = spacy.load('en_core_web_sm')

# Apply NER on a sample text
sample_text = combined_df['text'].iloc[0]
doc = nlp(sample_text)

# Extract and display entities
for entity in doc.ents:
    print(f"Entity: {entity.text}, Label: {entity.label_}")


### Part-of-Speech Tagging:
POS tagging is the process of assigning a grammatical category (such as noun, verb, adjective, etc.) to each word in a sentence. Each word is tagged based on its role in the sentence. POS tagging helps understand the structure and meaning of sentences by identifying how words function within them. 

In [ ]:
# Tokenize and POS tag using spaCy
doc = nlp(sample_text)

# Extract POS tags
for token in doc:
    print(f"Word: {token.text}, POS: {token.pos_}, Tag: {token.tag_}")


### Sentiment Analysis:
Sentiment analysis determines the emotional tone behind a piece of text. It categorizes the text as positive, negative, or neutral, providing insights into opinions or attitudes expressed. 

Polarity (0.0903): This value ranges from -1 to 1, where -1 indicates a negative sentiment, 1 indicates a positive sentiment, and 0 represents a neutral sentiment. In this case, a polarity of 0.0903 suggests a slightly positive sentiment, but it's close to neutral.

Subjectivity (0.3806): This value ranges from 0 to 1, where 0 indicates an objective statement (factual) and 1 represents a subjective statement (opinion-based). Here, the subjectivity score of 0.3806 suggests that the text is more objective but has some level of subjectivity.

In [ ]:
!pip install textblob

In [ ]:
!python -m textblob.download_corpora


In [ ]:
from textblob import TextBlob

# Apply sentiment analysis on a sample text
blob = TextBlob(sample_text)
print(f"Sentiment Polarity: {blob.sentiment.polarity}, Subjectivity: {blob.sentiment.subjectivity}")

### WordCloud: 
Visualize the most common words in both human-generated and machine-generated texts.

In [ ]:
!pip3 install wordcloud matplotlib

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Combine human-generated and machine-generated texts
human_texts = combined_df[combined_df['label'] == 'HGT']['cleaned_text']
machine_texts = combined_df[combined_df['label'] == 'MGT']['cleaned_text']

# Join the texts to form a single string for each category
human_text_combined = ' '.join(human_texts)
machine_text_combined = ' '.join(machine_texts)

# Create a function to generate word cloud
def generate_word_cloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=20)
    plt.show()

# Generate word clouds for human-generated text
generate_word_cloud(human_text_combined, 'Word Cloud for Human-Generated Text')

# Generate word clouds for machine-generated text
generate_word_cloud(machine_text_combined, 'Word Cloud for Machine-Generated Text')


# HyperLink analysis

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Function to retrieve and clean text from a URL
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            texts = soup.find_all(text=True)
            blacklist = ['style', 'script', 'head', 'title', 'meta', '[document]']
            visible_texts = filter(lambda t: t.parent.name not in blacklist, texts)
            text_content = u" ".join(t.strip() for t in visible_texts)
            text_content = re.sub(r'\s+', ' ', text_content)  # Clean extra whitespace
            return text_content
        else:
            print(f"Failed to retrieve URL content. Status Code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [ ]:
# Example: Extract text from a News Article
url = "https://www.bbc.com/news/technology-56933733"  # Replace this with an actual URL for testing
extracted_text = extract_text_from_url(url)

In [ ]:
# Preprocessing function (extend as needed based on your notebook's logic)
def preprocess_text(text):
    cleaned_text = text.lower()  # Example preprocessing (lowercasing)
    return cleaned_text


In [ ]:
# Preprocess the extracted text only if it's not None
if extracted_text:
    preprocessed_text = preprocess_text(extracted_text)
    print(f"Preprocessed Text: {preprocessed_text}")
else:
    print("No text extracted from the URL. Cannot preprocess.")

In [ ]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=500)),
    ('classifier', RandomForestClassifier(n_estimators=50, max_depth=5))
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
# After fitting the pipeline, we do the prediction
if preprocessed_text:
    prediction = pipeline.predict([preprocessed_text])

    # Interpret and print the result
    result = "Human Generated Text (HGT)" if prediction == 0 else "Machine Generated Text (MGT)"
    print(f"Prediction: {result}")
else:
    print("No text extracted from the URL to process.")